In [1]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

%matplotlib widget

In [2]:
# cluster plotting function

def plot_clusters(band_num):
    
    plt.figure(figsize=(10,3))
    
#     mean vs sigma
    plt.subplot(1, 3, 1)
    plt.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Variance")
    plt.legend()
    
#     sigma vs beta
    plt.subplot(1, 3, 2)
    plt.scatter(orig_beta_feat[:,band_num], orig_sigma_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_beta_feat[:,band_num], laser_sigma_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Beta")
    plt.ylabel("Variance")
    plt.legend()
     
#     beta vs mean
    plt.subplot(1, 3, 3)
    plt.scatter(orig_mu_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    plt.scatter(laser_mu_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    plt.xlabel("Mean")
    plt.ylabel("Beta")
    plt.legend()
    
    plt.suptitle("Laser vs Original for {}th Band".format(band_num))
    
    fig = plt.figure(figsize=(12,6))
#     plt.figure(figsize=(20,6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(orig_mu_feat[:,band_num], orig_sigma_feat[:,band_num], orig_beta_feat[:,band_num], marker='o', label="Original")
    ax.scatter(laser_mu_feat[:,band_num], laser_sigma_feat[:,band_num], laser_beta_feat[:,band_num], marker='x', label="Laser")
    ax.set_xlabel('Mean')
    ax.set_ylabel('Variance')
    ax.set_zlabel('Beta')
    ax.legend()

In [3]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files.sort()
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_009.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Laser/laser_010.mp3']
10


In [4]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files.sort()
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_000.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_001.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_002.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_003.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_004.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_005.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_006.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_007.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_008.mp3', '/home/hashim/PHD/audio_data/Laser_Original_New/Original/original_009.mp3']
10


# Wavelet Packet Analysis starts from here

In [5]:
%%capture
# compute coefficients for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []

for lf in laser_files:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for k in keys:
        
        cf = wp[k].data
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)

#     laser_spec.append(S_db)

# laser_spec = np.array(laser_spec)
# laser_spec

In [6]:
print(laser_mu_feat.shape)
print(laser_sigma_feat.shape)
print(laser_beta_feat.shape)

(10, 32)
(10, 32)
(10, 32)


In [7]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    

    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for k in keys:
        
        cf = wp[k].data
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)


In [8]:
print(orig_mu_feat.shape)
print(orig_sigma_feat.shape)
print(orig_beta_feat.shape)

(10, 32)
(10, 32)
(10, 32)


In [9]:
aprox_detail_coeff = 0 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
aprox_detail_coeff = 1 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
aprox_detail_coeff = 2 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
aprox_detail_coeff = 3 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
aprox_detail_coeff = 4 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
aprox_detail_coeff = 5 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
aprox_detail_coeff = 6 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
aprox_detail_coeff = 7 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
aprox_detail_coeff = 8 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
aprox_detail_coeff = 9 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
aprox_detail_coeff = 10 
plot_clusters(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 11 
plot_clusters(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 12 
plot_clusters(aprox_detail_coeff)

In [ ]:
aprox_detail_coeff = 15 
plot_clusters(aprox_detail_coeff)

In [23]:
aprox_detail_coeff = 18 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
aprox_detail_coeff = 21 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
aprox_detail_coeff = 25 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
aprox_detail_coeff = 27 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
aprox_detail_coeff = 30 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#  Below Results are for the VAD audios 

In [28]:
laser_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Laser_cleaned/'
laser_files_vad = list(glob.glob(os.path.join(laser_dir, '*.mp3')))
laser_files_vad.sort()
# print(laser_files)
print(len(laser_files_vad))

72


In [29]:
orig_dir = '/home/hashim/PHD/audio_data/Laser_Original_New/Original_cleaned/'
orig_files_vad = list(glob.glob(os.path.join(orig_dir, '*.mp3')))
orig_files_vad.sort()
# print(orig_files)
print(len(orig_files_vad))

74


In [30]:
%%capture
# compute spectograms for both laser and original
laser_spec = []

laser_mu_feat = []
laser_sigma_feat = []
laser_beta_feat = []

for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    
    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for k in keys:
        
        cf = wp[k].data
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]
    
#         print(f_param)
#         print(f.summary())

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    laser_mu_feat.append(mu_feat)
    laser_sigma_feat.append(sigma_feat)
    laser_beta_feat.append(beta_feat)    
    
laser_mu_feat = np.array(laser_mu_feat)
laser_sigma_feat = np.array(laser_sigma_feat)
laser_beta_feat = np.array(laser_beta_feat)

In [31]:
%%capture
orig_spec = []

orig_mu_feat = []
orig_sigma_feat = []
orig_beta_feat = []
for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    

    mu_feat = []
    sigma_feat = []
    beta_feat = []
    for k in keys:
        
        cf = wp[k].data
        
        f = Fitter(cf, distributions=["gennorm"])

        f.fit()
#         print(f.get_best(method = 'sumsquare_error'))

        f_param = f.fitted_param["gennorm"]

        mu_feat.append(f_param[1])
        sigma_feat.append(f_param[2])
        beta_feat.append(f_param[0])
        
    orig_mu_feat.append(mu_feat)
    orig_sigma_feat.append(sigma_feat)
    orig_beta_feat.append(beta_feat)
    
orig_mu_feat = np.array(orig_mu_feat)
orig_sigma_feat = np.array(orig_sigma_feat)
orig_beta_feat = np.array(orig_beta_feat)


In [32]:
aprox_detail_coeff = 0 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
aprox_detail_coeff = 2 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
aprox_detail_coeff = 3 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
aprox_detail_coeff = 6 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
aprox_detail_coeff = 9 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
aprox_detail_coeff = 14 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
aprox_detail_coeff = 17 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
aprox_detail_coeff = 24 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
aprox_detail_coeff = 27 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
aprox_detail_coeff = 31 
plot_clusters(aprox_detail_coeff)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaaaa Laser

In [42]:
# laser histograms of approximation coefficients
nrows = int(len(laser_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[0]].data, bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaaaa Original

In [43]:
# Original histograms of Approximation coefficients
nrows = int(len(orig_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[0]].data, bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaaad Laser

In [128]:
# laser histograms of approximation coefficients
nrows = int(len(laser_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[1]].data, bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaaad Original


In [129]:
# Original histograms of Approximation coefficients
nrows = int(len(orig_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[1]].data, bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaada Laser

In [130]:
# laser histograms of approximation coefficients
nrows = int(len(laser_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[2]].data, bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaada Original

In [131]:
# Original histograms of Approximation coefficients
nrows = int(len(orig_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[2]].data, bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaadd Laser 

In [132]:
# laser histograms of approximation coefficients
nrows = int(len(laser_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for lf in laser_files_vad:
    audio_data, sr = librosa.load(lf, res_type='kaiser_fast')
#     coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)

    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[3]].data, bins='auto')
#     ax[i+1].hist(coeffs[-1], bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# aaadd Original

In [133]:
# Original histograms of Approximation coefficients
nrows = int(len(orig_files_vad)/2)
fig, axes = plt.subplots(nrows, 2, figsize=(12,20))

ax = axes.flatten()
i = 0
for of in orig_files_vad:
    audio_data, sr = librosa.load(of, res_type='kaiser_fast')
    wp = pywt.WaveletPacket(data=audio_data, wavelet='db1', mode='symmetric', maxlevel=5)

    keys = [node.path for node in wp.get_level(5, 'freq')]
    
    ax[i].hist(wp[keys[3]].data, bins='auto')
        
    i+=1

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …